# P5-04: Prediction and post-processing of the masks
Post-processing with two versions:
- Contour code, standard library
- Chain code in python

## Set up Google Colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/BA_SemanticSegmentation_JonasHeinke/___P7_Eigenes_Projekt_Segmentation_QU/

/content/drive/.shortcut-targets-by-id/1i0GOg0AT_UuZJJUMz0D46PD2S2VfYbPB/NN_Segmentation/BA_Semantische_Segmentation/_Skript_Projekte


In [ ]:
ls

Codeoptimierung_Hinweise.py
___Datasets/
___P3-1_UNET-RESNET_NODE-PL-Segmenation_QU/
___P3_neural-odes-segmentation-master_QU/
___P4-1_UNET-PL_Segmentation_CRAG/
___P4_neural-odes-segmentation-master_CRAG/
___P5_Eigenes_Projekt_Segmentation_CRAG/
___P7_Eigenes_Projekt_Segmentation_QU/


In [ ]:
cd ___P7_Eigenes_Projekt_Segmentation_QU/

/content/drive/.shortcut-targets-by-id/1i0GOg0AT_UuZJJUMz0D46PD2S2VfYbPB/NN_Segmentation/BA_Semantische_Segmentation/_Skript_Projekte/___P7_Eigenes_Projekt_Segmentation_QU


## Libraries, modules

In [ ]:
# Imports
import pathlib
import numpy as np
import torch
from skimage.io import imread
from skimage.transform import resize
import os
import matplotlib.pyplot as plt
from PIL import Image
import pickle # zum speichern
#------------------------------------------------------------------
from  MODULE.PL.metrics import ObjectDice, ObjectHausdorff, F1score
#---------------
#from MODULE.JS.inference import predict
from MODULE.JS.transformations import normalize_01, re_normalize
from MODULE.JS.unet import UNet
#------------
# Konfigurationsdaten
from configuration_QU import Path   as PATH   # Pfade und Dateinamen
from configuration_QU import Inputs as INPUT    # Imageparameter
from configuration_QU import CfgModel as CFG_MODEL  # Modellparameter
from configuration_QU import Postprocess as POST  # Nachbearbeitung
from configuration_QU import EXPERIMENT          # Name des Experiments
# Transformationen und Anpassungen
from MODULE.JH.img_array_transform import ArrayTransform as TRANSFORM
# Bildoperationen, Kettencode
import MODULE.JH.image_processing as IP
# Prognose mit Anpassungen
from MODULE.JH.prediction import Prediction as PREDICTION
# View Bild-Masken-Sets
from MODULE.JH.visualize import Show as SHOW
# Instanzen von Klassen ggf.
transform=TRANSFORM() # Klassenistanz

In [ ]:
# Printkontrolle
VERBOSE=True
# Experiment, Daten aus dem Training
print(EXPERIMENT)

QU_vgl(open8x8_2021-08-26)_blocks7_cout2_optAdam_lr0.001_ep600_h512_w768_ft64


## List of file paths of the test data

In [ ]:
path=PATH() # Instanz der Klasse für Methodenaufruf erforderlich
path_images=path.testimages
path_masks=path.testmasks
# Source path of the trained model.
path_model_experiment=path.model / EXPERIMENT
# Destination path for results of the experiment
path_result_experiment   = path.results  / EXPERIMENT
if not os.path.exists(path_result_experiment):
    os.mkdir(path_result_experiment)
# Filter
imgfilter= ['train_??.bmp',       'testA_??.bmp',     'testB_??.bmp']
maskfilter=['train_??_anno.bmp', 'testA_??_anno.bmp', 'testB_??_anno.bmp']
fidx=1
if path.serie=='B':
    fidx=2
# input and target files -TRAIN[0], TESTA[1], TESTB[2]
image_filenames = path.get_filenames(path_images, dateifilter= imgfilter[fidx],  sort=True)
mask_filenames  = path.get_filenames(path_masks,  dateifilter= maskfilter[fidx], sort=True)
if VERBOSE:
    print('Serie: ', path.serie)
    print(path_images)
    print(path_masks)

Serie:  A
/content/drive/.shortcut-targets-by-id/1i0GOg0AT_UuZJJUMz0D46PD2S2VfYbPB/NN_Segmentation/BA_Semantische_Segmentation/_Skript_Projekte/___P7_Eigenes_Projekt_Segmentation_QU/Warwick QU Dataset (Released 2016_07_08)
/content/drive/.shortcut-targets-by-id/1i0GOg0AT_UuZJJUMz0D46PD2S2VfYbPB/NN_Segmentation/BA_Semantische_Segmentation/_Skript_Projekte/___P7_Eigenes_Projekt_Segmentation_QU/Warwick QU Dataset (Released 2016_07_08)


In [ ]:
print('Anzahl der Bild-Masken-Paare (Samples) für Prognose: ',len(image_filenames),' : ', len(mask_filenames))
if VERBOSE:
    for idx in range(len(image_filenames)):
        print(idx, ' | ', os.path.basename(image_filenames[idx]),'\t-> ', os.path.basename(mask_filenames[idx]))

Anzahl der Bild-Masken-Paare (Samples) für Prognose:  60  :  60
0  |  testA_01.bmp 	->  testA_01_anno.bmp
1  |  testA_02.bmp 	->  testA_02_anno.bmp
2  |  testA_03.bmp 	->  testA_03_anno.bmp
3  |  testA_04.bmp 	->  testA_04_anno.bmp
4  |  testA_05.bmp 	->  testA_05_anno.bmp
5  |  testA_06.bmp 	->  testA_06_anno.bmp
6  |  testA_07.bmp 	->  testA_07_anno.bmp
7  |  testA_08.bmp 	->  testA_08_anno.bmp
8  |  testA_09.bmp 	->  testA_09_anno.bmp
9  |  testA_10.bmp 	->  testA_10_anno.bmp
10  |  testA_11.bmp 	->  testA_11_anno.bmp
11  |  testA_12.bmp 	->  testA_12_anno.bmp
12  |  testA_13.bmp 	->  testA_13_anno.bmp
13  |  testA_14.bmp 	->  testA_14_anno.bmp
14  |  testA_15.bmp 	->  testA_15_anno.bmp
15  |  testA_16.bmp 	->  testA_16_anno.bmp
16  |  testA_17.bmp 	->  testA_17_anno.bmp
17  |  testA_18.bmp 	->  testA_18_anno.bmp
18  |  testA_19.bmp 	->  testA_19_anno.bmp
19  |  testA_20.bmp 	->  testA_20_anno.bmp
20  |  testA_21.bmp 	->  testA_21_anno.bmp
21  |  testA_22.bmp 	->  testA_22_anno.bmp


## Read in images and masks
- The size of the test images and test masks correspond to the training. Scaling is done accordingly.

In [ ]:
# read images and store them in memory
images = [imread(img_name) for img_name in image_filenames]
actual_masks = [imread(mask_name) for mask_name in mask_filenames]
# Resize images and targets
images_res = [resize(img, (INPUT.h_res, INPUT.w_res, INPUT.c_res)) for img in images]
resize_kwargs = {'order': 0, 'anti_aliasing': False, 'preserve_range': True}
actual_masks_res = [resize(mask, (INPUT.h_res, INPUT.w_res), **resize_kwargs) for mask in actual_masks]

# Anzahl der Stichproben
sample_anzahl=len(images)

## Set up the model
Prediction parameters must correspond to the training parameters. Common configuration supports this process.

In [ ]:
# Determination of the available device (CPU, GPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model
model = UNet(in_channels=INPUT.c_res,
             out_channels= CFG_MODEL.c_out,
             n_blocks=CFG_MODEL.n_blocks,
             start_filters=CFG_MODEL.ft,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).to(device)
model_weights = torch.load(path_model_experiment/ 'model')
model.load_state_dict(model_weights)

RuntimeError: ignored

In [ ]:
# Prediction of the segmentation mask
prediction=PREDICTION(model, device, True)
predict_masks = [prediction.mask(img) for img in images_res]

## Evaluation of the prognosis of the test set
- Here just for checking purposes, see project "P7-05 result visualization"

In [ ]:
transform=TRANSFORM()
dice, hausdorff, f1, dice_full = 0, 0, 0, 0
i_error=0
anzahl=0
# Table header
print_string=(f' idx | dice_img  | f1_img    | hausdorff_img | result-filename | input-filename')
print(print_string)
for idx in range(sample_anzahl):
    try:
        predict_mask_two= transform.twoClasses(predict_masks[idx])
        actual_masks_two= transform.twoClasses(actual_masks_res[idx])
        dice_img = ObjectDice(predict_mask_two,        actual_masks_two)
        f1_img = F1score(predict_mask_two,              actual_masks_two)
        hausdorff_img = ObjectHausdorff(predict_mask_two,  actual_masks_two)
        dice += dice_img
        f1 += f1_img
        hausdorff += hausdorff_img
        print_string=(f' {idx:3d} | {dice_img:9.3f} | {f1_img:9.3f} | {hausdorff_img:13.3f} | result_test_{idx:-02d}  | {os.path.basename(mask_filenames[idx])}')
        print(print_string)
        anzahl +=1
    except:
        i_error +=1
        print('Error: ',i_error, 'Cycle: ', idx)


In [ ]:
print('-- Mean values ------------------------------------')
print('ObjectDice:', dice / anzahl )
print('F1:', f1 / anzahl )
print('Weighted shape:', hausdorff / anzahl)
print('Number io.: ', anzahl)
print('Errors: ', i_error)


## Post-processing of the predicted masks (-> object masks) and code to outline the predicted glands and their identification
Two alternative variants
- Remove small objects
- Separate objects as far as possible (binary intensity morphology)
- Outline
- Set IDs

Binary intensity morphology with module: scipy.ndimage
- Separating objects.
- Can only differentiate between two states (background and object).
- But can emphasize a very specific object and separate it from one another.
Konur code with module: skimage.measure
- Can outline objects and assign object-specific IDs
- Empty areas of the objects can be filled

In [ ]:
from scipy import ndimage
from importlib import reload
import MODULE.JH.image_processing
reload(MODULE.JH.image_processing)
import MODULE.JH.image_processing as IP

In [ ]:
# Lists
contour_codes_list=[]
# object_filled_list=[]
predict_masks_morph_list=[]
object_array_list=[]

In [ ]:
# VARIANT 2 - contour code (library / module: skimage.measure)
# Identification of the objects, variant 2
if POST.ident=='measure':
    # Fill holes
    idObjects= IP.ObjectIDsOfArray()
    for idx in range(sample_anzahl):
        mask_morph=idObjects.fill_objects(predict_masks[idx], POST.opening_structure)
        contur_codes, object_array=idObjects.getObjects(mask_morph)
        predict_masks_morph_list.append(mask_morph)
        object_array_list.append(object_array)
        contour_codes_list.append(contur_codes) 



In [ ]:
convert=IP.Convert() # Instance to convert
# VARIANT 1: CHAINCODE (Python)
if POST.ident=='chain':
    for idx in range(sample_anzahl):
        mask_morph=ndimage.binary_opening(predict_masks[idx], structure=np.ones(POST.opening_structure)).astype(int)
        # -> Returns: 0 = no object or 1 = object
        print('\n--- Mask idx: ', idx)
        idObject= IP.IdentifyObject(mask_morph, False) # Klasseninstanz MODUL
        # class-id - areas with this id are processed
        # id_ - The elements of the area receive this id
        chaincodes, object_array, fill_array=idObject.chaincode(class_id=1, id_=1)       # Klassenmethode
        # Converts chain code to contour array
        contour_codes=convert.chains_to_contourcodes_2(chaincodes)
        predict_masks_morph_list.append(mask_morph)
        contour_codes_list.append(contour_codes)
        object_array_list.append(object_array) 

## Save object arrays and contour codes

In [ ]:
# Control dimensions
if VERBOSE:
    print('Input images:   ', images_res[0].shape)
    print('Actual masks:  ',  actual_masks_res[0].shape)
    print('Prediction masks: ', predict_masks[0].shape)
    print('Path of model: ', path_model_experiment)
    print('Path of result: ', path_result_experiment)

# Verzeichnisse zum Abspeichern der Bilder vorbereiten
path_inputImages = path_result_experiment / 'images'    # Bilder skaliert entsprechend Trainingsvorgabe
path_actualMasks= path_result_experiment / 'actualMasks'        # Tatsächliche Masken
path_predictMasks= path_result_experiment / 'predictMasks'      # Prognostizierte Masken
path_predictMasksMorph= path_result_experiment / 'predictMasksMorph'          # mit opening (morph)
path_predictObjects= path_result_experiment / 'predictObjects'  # Nachbearbeitete prognostizierte Masken
path_predictContourCodes = path_result_experiment / 'predictContourCodes' # Kontur der Objekte als Kettencode
# New subdirectories    
os.chdir(path_result_experiment)
os.mkdir(path_inputImages)
os.mkdir(path_actualMasks)
os.mkdir(path_predictMasks)
os.mkdir(path_predictMasksMorph)
os.mkdir(path_predictObjects)
os.mkdir(path_predictContourCodes)

In [ ]:
from importlib import reload
import MODULE.JH.visualize
reload(MODULE.JH.visualize)
from MODULE.JH.visualize import Show as SHOW
show=SHOW(figsize=(18,25))

In [ ]:
print(len(contour_codes_list)) # Listenelemente
print(len(contour_codes_list[0])) # konturen drs Arrays 0'

In [ ]:
idx_list=list([25,29,23])
#idx_list=list([10,11,12,13])
listset=list([images,  actual_masks,         predict_masks, predict_masks_morph_list,  object_array_list])
titles=list(['Image', 'Aktual mask', 'Predicted mask', 'with morphology',    'IDs - identify'])
path_set= path_result_experiment  / f'images_masks_listset_(P7-04)_{str(idx_list)}.png'
show.list_set(idx_list, listset, titles, path=path_set)

## Save the results

In [ ]:
# Images, Masken und Ergebnsimasken der Testbilder speichern
for idx in range(sample_anzahl):
    print('idx:', idx, end=', ')
    # Alle Dateien haben den gleichen Dateinamen,
    # werden aber in unterschiedlichen Verzeichnissen gespeichert
    file_name=f'result_test_{idx:02d}.png'
    # Portieren von ndarray nach *.png
    input_image         = Image.fromarray((images_res[idx]*255).astype(np.uint8))
    actual_mask_img     = Image.fromarray((actual_masks_res[idx]).astype(np.uint8))
    predict_mask_img    = Image.fromarray((predict_masks[idx]).astype(np.uint8))
    predict_masks_morph_img = Image.fromarray((predict_masks_morph_list[idx]).astype(np.uint8))
    object_image        = Image.fromarray((object_array_list[idx]).astype(np.uint8))
    
    # Speichern der Arrays=Bilder einschließlich Zwischenergebnisse
    input_image.save(path_inputImages / file_name)
    actual_mask_img.save(path_actualMasks / file_name)
    predict_mask_img.save(path_predictMasks / file_name)
    predict_masks_morph_img.save(path_predictMasksMorph / file_name)
    object_image.save(path_predictObjects / file_name)
    # Code der Konturen 
    file_name=f'result_test_{idx:02d}.pkl'
    file_codes=open(path_predictContourCodes / file_name, 'wb')
    pickle.dump(contour_codes_list[idx], file_codes)
    file_codes.close()

In [ ]:
print('ENDE')